In [ ]:
import pandas            as pd
import missingno         as msno
import seaborn           as sns
import matplotlib.pyplot as plt
import statsmodels.api   as sm
import scipy.stats       as stats

# Data Wrangling

In [ ]:
# Import dataset into Pandas DataFrame
df_raw    = pd.read_csv("../dat/ship_data.csv")
n_records = df_raw.shape[0]

# Check column names and data types
df_raw.info()

### Missing Records

In [ ]:
# Check missing values
print(df_raw.isnull().sum())
msno.matrix(df_raw)

In [ ]:
# Drop rows with missing target (Main Engine Fuel Consumption) ~ 1% of records
df_mod = df_raw.copy().dropna()
df_mod = df_mod.reset_index().drop('index',axis=1)
print('Percentage of missing records:   ', ((1 - df_mod.shape[0] / n_records) * 100))
print('Percentage of remaining records: ', ((    df_mod.shape[0] / n_records) * 100))
msno.matrix(df_mod)

### Time Series Visualization

In [ ]:
# Create helper sublists of column names
cols_main  = df_mod.columns[0:4]
cols_draft = df_mod.columns[4:8]
cols_shaft = df_mod.columns[8:11]
cols_speed = df_mod.columns[11:15]
cols_wind  = df_mod.columns[15:19]
cols_sea   = df_mod.columns[19:23]
cols_wave  = df_mod.columns[23:26]
df_mod.shape

In [ ]:
# Time and main engine
fig_main, axes_main = plt.subplots(len(cols_main),1, figsize=(16,len(cols_main)*1.5), sharex=True)
df_mod[cols_main].plot(subplots=True, ax=axes_main)

In [ ]:
# Draft sensors
fig_draft, axes_draft = plt.subplots(len(cols_draft),1, figsize=(16,len(cols_draft)*1.5), sharex=True)
df_mod[cols_draft].plot(subplots=True, ax=axes_draft)

In [ ]:
# Shaft performance
fig_shaft, axes_shaft = plt.subplots(len(cols_shaft),1, figsize=(16,len(cols_shaft)*1.5), sharex=True)
df_mod[cols_shaft].plot(subplots=True, ax=axes_shaft)

In [ ]:
# Vessel speed
fig_speed, axes_speed = plt.subplots(len(cols_speed),1, figsize=(16,len(cols_speed)*1.5), sharex=True)
df_mod[cols_speed].plot(subplots=True, ax=axes_speed)

In [ ]:
# Wind conditions
fig_wind, axes_wind = plt.subplots(len(cols_wind),1, figsize=(16,len(cols_wind)*1.5), sharex=True)
df_mod[cols_wind].plot(subplots=True, ax=axes_wind)

In [ ]:
# Sea conditions
fig_sea, axes_sea = plt.subplots(len(cols_sea),1, figsize=(16,len(cols_sea)*1.5), sharex=True)
df_mod[cols_sea].plot(subplots=True, ax=axes_sea)

In [ ]:
# Wave conditions
fig_wave, axes_wave = plt.subplots(len(cols_wave),1, figsize=(16,len(cols_wave)*1.5), sharex=True)
df_mod[cols_wave].plot(subplots=True, ax=axes_wave)

### Data Cleaning

In [ ]:
# Convert Unix time to datetime format
df_mod['Time'] = pd.to_datetime(df_mod['Time'], unit='s')
df_mod['Time'].head()

In [ ]:
# Calibrate fuel consumption with most frequent near-zero measurement (-0.048 MT/day)
print(df_mod[cols_main[1]].value_counts().head())
df_mod[cols_main[1]] -= df_mod[cols_main[1]].value_counts().index[0]

# Drop records with negative fuel consumption ~ 16.2% of records
cutoff = 0.0
print('Percentage of records with negative consumption: ',
      df_mod[df_mod[cols_main[1]] < cutoff].shape[0] / n_records * 100)
df_mod = df_mod.drop(df_mod[df_mod[cols_main[1]] < cutoff].index)
df_mod = df_mod.reset_index().drop('index',axis=1)
print('Percentage of remaining records:                 ', ((df_mod.shape[0] / n_records) * 100))

In [ ]:
# Revert HFO and MGO to booleans (unbalanced value counts)
print(df_mod[cols_main[2]].value_counts())
df_mod[cols_main[2]] = df_mod[cols_main[2]].astype('int64')
df_mod[cols_main[3]] = df_mod[cols_main[3]].astype('int64')

# Check that only one fuel gauge is indicated at a time 
df_mod['Fuel Gauge'] = df_mod[cols_main[2]] + df_mod[cols_main[3]]
print(df_mod['Fuel Gauge'].value_counts())

# Drop records with neither HFO nor MGO indicator ~ 1.1% of records
print('Percentage of records with no fuel indicator: ',
      df_mod[df_mod['Fuel Gauge'] == 0].shape[0] / n_records * 100)
df_mod = df_mod.drop(df_mod[df_mod['Fuel Gauge'] == 0].index)
df_mod = df_mod.reset_index().drop('index',axis=1)
print('Percentage of remaining records:              ', ((df_mod.shape[0] / n_records) * 100))

In [ ]:
# Make rudder data continuous by using -180 to 180 degree angles
degrees = 360
print(df_mod[cols_speed[3]].value_counts().sort_index(ascending=False).head())
df_mod.loc[df_mod[cols_speed[3]] > 180, cols_speed[3]] -= degrees

In [ ]:
# Replace 0.0 apparent wind speed records with median
print(df_mod[cols_wind[0]].value_counts().sort_index().head())
df_mod.loc[df_mod[cols_wind[0]] == 0, cols_wind[0]] = df_mod[df_mod[cols_wind[0]] > 0][cols_wind[0]].median()

In [ ]:
# Replace 0.0 apparent wind direction records with median
print(df_mod[cols_wind[1]].value_counts().head())
df_mod.loc[df_mod[cols_wind[1]] == 0, cols_wind[1]] = df_mod[df_mod[cols_wind[1]] > 0][cols_wind[1]].median()

In [ ]:
# Replace 0.0 true wind speed records with median
print(df_mod[cols_wind[2]].value_counts().head())
df_mod.loc[df_mod[cols_wind[2]] == 0, cols_wind[2]] = df_mod[df_mod[cols_wind[2]] > 0][cols_wind[2]].median()

In [ ]:
# Replace 180.0 true wind direction records with median
print(df_mod[cols_wind[3]].value_counts().head())
df_mod.loc[df_mod[cols_wind[3]] == 180, cols_wind[3]] = df_mod[df_mod[cols_wind[3]] != 180][cols_wind[3]].median()

In [ ]:
# Replace non-positive temperature records with median
print(df_mod[cols_sea[0]].value_counts().sort_index().head())
df_mod.loc[df_mod[cols_sea[0]] <= 0, cols_sea[0]] = df_mod[df_mod[cols_sea[0]] > 0][cols_sea[0]].median()

In [ ]:
# Replace 0.0 sea current direction records with median
print(df_mod[cols_sea[1]].value_counts().head())
df_mod.loc[df_mod[cols_sea[1]] == 0, cols_sea[1]] = df_mod[df_mod[cols_sea[1]] > 0][cols_sea[1]].median()

In [ ]:
# Replace 0.0 sea current speed records with median
print(df_mod[cols_sea[2]].value_counts().head())
df_mod.loc[df_mod[cols_sea[2]] == 0, cols_sea[2]] = df_mod[df_mod[cols_sea[2]] > 0][cols_sea[2]].median()

In [ ]:
# Replace 0.0 wave height records with median
print(df_mod[cols_wave[0]].value_counts().head())
df_mod.loc[df_mod[cols_wave[0]] == 0, cols_wave[0]] = df_mod[df_mod[cols_wave[0]] > 0][cols_wave[0]].median()

In [ ]:
# Replace 0.0 wave period records with median
print(df_mod[cols_wave[1]].value_counts().head())
df_mod.loc[df_mod[cols_wave[1]] == 0, cols_wave[1]] = df_mod[df_mod[cols_wave[1]] > 0][cols_wave[1]].median()

In [ ]:
# Replace 0.0 wave direction records with median
print(df_mod[cols_wave[2]].value_counts().head())
df_mod.loc[df_mod[cols_wave[2]] == 0, cols_wave[2]] = df_mod[df_mod[cols_wave[2]] > 0][cols_wave[2]].median()

### Descriptive Statistics

In [ ]:
# Time and main engine
df_mod[cols_main].describe()

In [ ]:
# Draft sensors
df_mod[cols_draft].describe()

In [ ]:
# Shaft performance
df_mod[cols_shaft].describe()

In [ ]:
# Vessel speed
df_mod[cols_speed].describe()

In [ ]:
# Wind conditions
df_mod[cols_wind].describe()

In [ ]:
# Sea conditions
df_mod[cols_sea].describe()

In [ ]:
# Wave conditions
df_mod[cols_wave].describe()

### Outliers

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_main].boxplot()

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_shaft[0:1]].boxplot()

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_speed[0:2]].boxplot()

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_speed[2:4]].boxplot()

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_wind].boxplot()
plt.xticks(rotation=5)

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_draft].boxplot()

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_sea].boxplot()
plt.xticks(rotation=5)

In [ ]:
# Check for outliers
plt.figure("Distribution", figsize=(16,4))
df_mod[cols_wave].boxplot()
plt.xticks(rotation=5)

In [ ]:
# Drop records with shaft speed less than 40 RPM ~ 0.64% of records
cutoff = 8
print(df_mod[df_mod[cols_speed[0]] <= cutoff].shape[0] / n_records * 100)
df_mod = df_mod.drop(df_mod[df_mod[cols_speed[0]] <= cutoff].index)
print(df_mod[cols_speed[0]].value_counts().sort_index().head())
df_mod[cols_speed[0]].hist(bins=30)

In [ ]:
# Drop records with fuel consumption in lowest quintile (less than 8 MT/day) ~ 40.6% of records
cutoff = 16
print(df_mod[df_mod[cols_main[1]] <= cutoff].shape[0] / n_records * 100)
df_mod = df_mod.drop(df_mod[df_mod[cols_main[1]] <= cutoff].index)
print(df_mod[cols_main[1]].value_counts().sort_index().head())
df_mod[cols_main[1]].hist(bins=30)

In [ ]:
# Drop records with shaft speed less than 40 RPM ~ 0.64% of records
cutoff = 40
print(df_mod[df_mod[cols_shaft[0]] <= cutoff].shape[0] / n_records * 100)
df_mod = df_mod.drop(df_mod[df_mod[cols_shaft[0]] <= cutoff].index)
print(df_mod[cols_shaft[0]].value_counts().sort_index().head())
df_mod[cols_shaft[0]].hist(bins=30)

In [ ]:
df_mod['Z Score'] = stats.zscore(df_mod[cols_main[1]], axis = 0)
df_mod['Z Score'].hist(bins=30)

In [ ]:
# Plot frequency distribution of fuel consumption
df_mod[cols_main[1]].hist(bins=30)

In [ ]:
# Plot frequency distribution of fuel consumption
df_mod[cols_speed[0]].hist(bins=30)

# Exploratory Data Analysis

In [ ]:
# Explore fuel consumption by fuel source
df_hfo = df_mod[df_mod[cols_main[2]] == 1].copy()
df_hfo['Fuel'] = 'Main Engine Using HFO'

df_mgo = df_mod[df_mod[cols_main[3]] == 1].copy()
df_mgo['Fuel'] = 'Main Engine Using MGO'

# Drop records with zero fuel consumption (for EDA only)
df_fuel = pd.concat([df_hfo, df_mgo])
df_fuel = df_fuel[df_fuel[cols_main[1]] > 0]

In [ ]:
# Print mean fuel consumption by fuel source
df_fuel.groupby('Fuel')[cols_main].median()

In [ ]:
# Plot fuel consumption by fuel source
plt.figure('Fuel Source', figsize=(16,4))
sns.boxplot(x='Fuel', y=cols_main[1], data=df_fuel)

In [ ]:
# Investigate multi-collinearity of shaft sensors (Power = Speed x Torque x unit conversion)
corr_shaft = df_mod[cols_shaft].corr()
corr_shaft

In [ ]:
# Investigate multi-collinearity of wind sensors
corr_wind = df_mod[cols_wind].corr()
corr_wind

In [ ]:
# Investigate multi-collinearity of speed and heading sensors
corr_speed = df_mod[cols_speed].corr()
corr_speed

# Feature Engineering

In [ ]:
# Engineer features for mean draft, trim (aft heavy), list and positive list (possible nonlinear relationship)
df_mod['Draft Mean (meters)'] = df_mod[cols_draft].mean(axis=1)
df_mod['Trim (meters)']       = df_mod[cols_draft[0]] - df_mod[cols_draft[1]]
df_mod['List (meters)']       = df_mod[cols_draft[2]] - df_mod[cols_draft[3]]
df_mod['Abs List (meters)']   = df_mod['List (meters)'].abs()

# Plot time series
fig_draft_fe, axes_draft_fe = plt.subplots(4,1, figsize=(16,4*1.5), sharex=True)
df_mod[['Draft Mean (meters)',
        'Trim (meters)',
        'List (meters)',
        'Abs List (meters)']].plot(subplots=True, ax=axes_draft_fe)

In [ ]:
# Engineer features for apparent sea current direction and speed
col         = 'Weather Service Apparent Sea Current Direction (degrees from bow)'
df_mod[col] = df_mod[cols_speed[2]] - df_mod[cols_sea[1]]

In [ ]:
# Plot time series
fig_sea_fe, axes_sea_fe = plt.subplots(3,1, figsize=(16,3*1.5), sharex=True)
df_mod[[cols_sea[0],
        cols_sea[3],
      'Weather Service Apparent Sea Current Direction (degrees from bow)']].plot(subplots=True, ax=axes_sea_fe)

# Clean Data

In [ ]:
# Reset DataFrame index after dropping records
df_mod = df_mod.reset_index().drop('index',axis=1)

# Regression Model

In [ ]:
df_regr = df_mod[[
    'Main Engine Fuel Consumption (MT/day)',
    'Main Engine Using MGO (bool)',
    'Draft Mean (meters)',
    'Trim (meters)',
    'List (meters)',
    'Abs List (meters)',
    'Shaft Speed (RPM)',
    'Shaft Torque (kNm)',
    'Speed Through Water (knots)',
    'Heading (degrees)',
    'Rudder Angle (degrees)',
    'Weather Service Apparent Wind Speed (knots)',
    'Weather Service Apparent Wind Direction (degrees from bow)',
    'Weather Service Temperature (celsius)',
    'Weather Service Sea Current Direction (degrees from north)',
    'Weather Service Sea Current Speed (knots)',
    'Water Depth (meters)',
    'Weather Service Apparent Sea Current Direction (degrees from bow)']]
df_regr.head()

In [ ]:
y = df_regr['Main Engine Fuel Consumption (MT/day)']
X = df_regr.drop('Main Engine Fuel Consumption (MT/day)',axis=1)
regr = sm.OLS(y,X).fit()
regr.summary()

In [ ]:
residuals = regr.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
fig.show()

# Speed Scores

# Sensor Drift